# First steps

## Generate bindings for functions

Using litgen is straightforward. The simplest use case is:

1. import litgen
2. instantiate some options
3. convert some code into C++ binding code and python declarations (stubs)
4. use the generated code

Below we import litgen, define the C++ code we want to bind, and run `litgen.generate_code`


In [1]:
# Import litgen
import litgen


# Instantiate some options
options = litgen.LitgenOptions()
# Code for which we will emit bindings
cpp_code = """
    // Mathematic operations

    // Adds two integers
    int Add(int x, int y = 2);

    int Sub(int x, int y = 2); // Substract two integers
    """
# Run the generator
generated_code = litgen.generate_code(options, cpp_code)

The generated code contains several elements:
- `stub_code` contains the python declarations, including all comments. They enable to have a flawless code navigation and completion inside IDEs.
- `pydef_code` contains the C++ binding code. This code is specific to pybind11
- `glue_code`: optional additional C++ code that is emitted in speficic advanced cases (more details later in this manual). Most of the time, it will be empty

Let's see the python declarations corresponding to the C++ code: they should be copied into a python stub file (*.pyi) that describes the bindings offered by your library.

>_Note: below, we import `litgen_demo` to be able to display code in this page. You will not need it: you can simply call `print(generated_code.stub_code)`_


In [2]:
from litgen.demo import litgen_demo

litgen_demo.demo(options, cpp_code, show_pydef=True)

# Below, you will see:
# - the original C++ code to the left
# - stub_code (i.e. the python stubs) to the right
# - pydef_code (i.e. the C++ binding code) at the bottom
# In this case, the glue code is empty 

## Binding a simple class or struct

For a simple class, public members and methods will be exposed. For a simple struct, a constructor with named parameters will be exposed in Python.

In [3]:
options = litgen.LitgenOptions()
cpp_code = """
class FooClass {
    private:
        int mPriv = 0;
    public:
        FooClass(int v);
        int mPublic = 1;
        void ShowInfo();
};

struct FooStruc {
    int a = 1;
    void ShowInfo();
};
"""

litgen_demo.demo(options, cpp_code, show_pydef=True)

## Generator options

There are numerous generations options that can be set, and they are explained in details later in this manual: See [full list](https://github.com/pthom/litgen/blob/main/src/litgen/options.py)

## About glue code

In some cases, some additional "glue code" will be emitted. This is the case for example when we want to bind a function that wants to modify a parameter whose type is immutable in python.

In this case, we will need to set one option (`options.fn_params_replace_modifiable_immutable_by_boxed__regex `)

In [4]:
options = litgen.LitgenOptions()
# This a regex which specifies that we want to adapt all functions with boxed types when needed
options.fn_params_replace_modifiable_immutable_by_boxed__regex = r".*"
cpp_code = """
    // changes the value of the bool parameter (passed by modifiable reference)
    // (This function will use a BoxedBool in the python code, so that its value can be modified)
    void SwitchBoolValue(bool &v);
    """
# Run the generator
litgen_demo.demo(options, cpp_code, show_pydef=True)